In [1]:
import pandas as pd
import geopandas as gpd
import fiona

In [2]:
fiona.listlayers('D:/JOB/IGEO/2020 - Metrogas - Geocoder/Prueba_CABA.gdb')

['CABA']

In [3]:
CABA = gpd.read_file("D:/JOB/IGEO/2020 - Metrogas - Geocoder/Prueba_CABA.gdb", driver='FileGDB', layer='CABA')
CABA.head()

,NOM_CALLE,NOM_CALLE_1,TIPO_VIA,NOM_LOCALIDAD,NOM_BARRIO,Join_Count,NOMCALLE,NOMPAR,DESDDER,DESDIZQ,...,TIPVIA,PAR_DER,PAR_IZQ,PARTIDO,Shape_Length,Ini_X,Ini_Y,Fin_X,Fin_Y,geometry
0,Francisco Pienovi,,Avenida,Pineyro,,0,None,None,NaN,NaN,...,None,Ciudad Aut. de Buenos Aires,Ciudad Aut. de Buenos Aires,None,85.595963,6.372743e+06,6.164134e+06,6.372737e+06,6.164219e+06,"MULTILINESTRING ((6372743.366 6164134.067, 637..."
1,Puente La Noria,None,Autopista,Villa Celina,,2,GENERAL PAZ BAJADA,CIUDAD DE BUENOS AIRES,NaN,NaN,...,AVENIDA,La Matanza,La Matanza,None,96.227761,6.366120e+06,6.159079e+06,6.366043e+06,6.159137e+06,"MULTILINESTRING ((6366119.823 6159079.067, 636..."
2,Puente La Noria,None,Autopista,Villa Celina,,1,GENERAL PAZ BAJADA,CIUDAD DE BUENOS AIRES,NaN,NaN,...,AVENIDA,Ciudad Aut. de Buenos Aires,Ciudad Aut. de Buenos Aires,None,92.067342,6.366098e+06,6.159206e+06,6.366173e+06,6.159152e+06,"MULTILINESTRING ((6366098.432 6159205.705, 636..."
3,Camino de la Ribera Norte,Coronel P. Aquino,Avenida,Villa Celina,,0,None,None,NaN,NaN,...,None,La Matanza,La Matanza,None,97.155845,6.366032e+06,6.159098e+06,6.366066e+06,6.159188e+06,"MULTILINESTRING ((6366032.072 6159098.465, 636..."
4,General Paz,None,Autopista,Villa Celina,,0,None,None,NaN,NaN,...,None,Ciudad Aut. de Buenos Aires,Ciudad Aut. de Buenos Aires,None,604.031699,6.365616e+06,6.159569e+06,6.366098e+06,6.159206e+06,"MULTILINESTRING ((6365616.088 6159568.977, 636..."


# Calculate Coordinates

In [ ]:
# for i in range(0,len(CABA)):    
#     CABA.loc[i,'Ini_X'] = CABA['geometry'][i][0].coords[0][0]
#     CABA.loc[i,'Ini_Y'] = CABA['geometry'][i][0].coords[0][1]
#     CABA.loc[i,'Fin_X'] = CABA['geometry'][i][0].coords[1][0]
#     CABA.loc[i,'Fin_Y'] = CABA['geometry'][i][0].coords[1][1]
# CABA.head()

# Null Values - Street Lenght

In [4]:
Direcctions = CABA.loc[(CABA.DESDDER.notnull()) & (CABA.DESDIZQ.notnull()) & (CABA.HASTDER.notnull()) & (CABA.HASTIZQ.notnull())]
Direcctions_Count = len(Direcctions)
print("Calles sin datos de altura:", len(CABA)-Direcctions_Count)
print("Calles con datos de altura:", Direcctions_Count)

Calles sin datos de altura: 8736
Calles con datos de altura: 20166


In [5]:
Calles = CABA.loc[CABA.DESDDER.notnull()]
NombreCalles = Calles.NOM_CALLE.unique()
print("Cantidad de calles:", len(NombreCalles))

Cantidad de calles: 1999


## Checking Street's Direction Number  

In [6]:
CABA['NullDir'] = 0
CABA.loc[CABA.DESDDER.isnull(), 'NullDir'] = 1
CABA.NullDir.value_counts()

0    20166
1     8736
Name: NullDir, dtype: int64

In [7]:
CallesAnalise = CABA

In [10]:
CABA['CheckAnt'] = 0
CABA['CheckSig'] = 0
CABA.loc[CABA.DESDDER.isnull(), 'CheckAnt'] = 5
CABA.loc[CABA.DESDDER.isnull(), 'CheckSig'] = 5
CABA.CheckAnt.value_counts()

0    20166
5     8736
Name: CheckAnt, dtype: int64

In [19]:
for l in range(0, len(CallesAnalise)):
    
    # Check Null
    NullValue = CallesAnalise.loc[l, 'NullDir']    
    
    # If is not a Null Value
    if(NullValue == 0):
        
        # Street Name and Numbers
        NombreDeCalle = CallesAnalise.loc[l, 'NOM_CALLE']
        DireccionInicio = CallesAnalise.loc[l, 'DESDIZQ']
        DireccionFin = CallesAnalise.loc[l, 'HASTDER']
        # Up and down street number
        DireccionAnt = DireccionInicio - 1
        DireccionSig = DireccionFin + 1
        
        # Get line coordinates
        Calle_Inicio_X = CallesAnalise.loc[l, 'Ini_X']
        Calle_Inicio_Y = CallesAnalise.loc[l, 'Ini_Y']
        Calle_Fin_X = CallesAnalise.loc[l, 'Fin_X']
        Calle_Fin_Y = CallesAnalise.loc[l, 'Fin_Y']
        
        # Search Down street 
        # If there´s a street
        if(len(CallesAnalise.loc[(CallesAnalise.NOM_CALLE == NombreDeCalle) & (CallesAnalise.HASTDER == DireccionAnt), 'Fin_X']) != 0):
            
            # null number value in down street 
            Calle_Value = list(CallesAnalise.loc[(CallesAnalise.NOM_CALLE == NombreDeCalle) & (CallesAnalise.HASTDER == DireccionAnt), 'NullDir'])[0]
            
            # if not null number direction
            if(Calle_Value == 0):
                # get end coordinates
                CalleAntX = list(CallesAnalise.loc[(CallesAnalise.NOM_CALLE == NombreDeCalle) & (CallesAnalise.HASTDER == DireccionAnt), 'Fin_X'])[0]
                CalleAntY = list(CallesAnalise.loc[(CallesAnalise.NOM_CALLE == NombreDeCalle) & (CallesAnalise.HASTDER == DireccionAnt), 'Fin_Y'])[0]
                
                # Coordinate Diference
                AntX_dif = abs(Calle_Inicio_X - CalleAntX)
                AntY_dif = abs(Calle_Inicio_Y - CalleAntY)
                
                # if down street end coord similar to actual street start coord => value = 0
                if((AntX_dif < 0.1) & (AntY_dif < 0.1)):
                    CallesAnalise.loc[l, 'CheckAnt'] = 0
                else:
                    CallesAnalise.loc[l, 'CheckAnt'] = 1    # if diferent => value = 1
            # if null down value, set check to 0 (no error)
            else:
                CallesAnalise.loc[l, 'CheckAnt'] = 0
            
        
        # If there´s no down street => value = 0
        else: 
            CallesAnalise.loc[l, 'CheckAnt'] = 0
            
        # Calculate Up street coordinates
        if(len(CallesAnalise.loc[(CallesAnalise.NOM_CALLE == NombreDeCalle) & (CallesAnalise.DESDIZQ == DireccionSig), 'Ini_X']) != 0):
            
            # null number value in up street 
            Calle_Value = list(CallesAnalise.loc[(CallesAnalise.NOM_CALLE == NombreDeCalle) & (CallesAnalise.DESDIZQ == DireccionSig), 'NullDir'])[0]
            
            # if not null number direction
            if(Calle_Value == 0):
                # Get Coord
                CalleSigX = list(CallesAnalise.loc[(CallesAnalise.NOM_CALLE == NombreDeCalle) & (CallesAnalise.DESDIZQ == DireccionSig), 'Ini_X'])[0]
                CalleSigY = list(CallesAnalise.loc[(CallesAnalise.NOM_CALLE == NombreDeCalle) & (CallesAnalise.DESDIZQ == DireccionSig), 'Ini_Y'])[0]

                # Coordinate Diference
                SigX_dif = abs(Calle_Fin_X - CalleSigX)
                SigY_dif = abs(Calle_Fin_Y - CalleSigY)
                
                # if down street end coord similar to actual street start coord => value = 0
                if((SigX_dif < 0.1) & (SigY_dif < 0.1)):
                    CallesAnalise.loc[l, 'CheckSig'] = 0
                else:
                    CallesAnalise.loc[l, 'CheckSig'] = 1    # if diferent => value = 1
            # if null up value, set check to 0 (no error)
            else:
                CallesAnalise.loc[l, 'CheckSig'] = 0
            
        # If there´s no up street => value = 0
        else:
            CallesAnalise.loc[l, 'CheckSig'] = 0   
        
    # Null values get value = 5
    else:
        CallesAnalise.loc[l, 'CheckAnt'] = 5
        CallesAnalise.loc[l, 'CheckSig'] = 5

In [20]:
CallesAnalise['Check'] = CallesAnalise['CheckAnt'] + CallesAnalise['CheckSig']
CallesAnalise.loc[CallesAnalise.Check > 6, 'Check'] = 5
CallesAnalise.Check.value_counts()

0    16550
5     8736
1     3046
2      570
Name: Check, dtype: int64

In [ ]:
# Coord == 
# 0    16354
# 5     8736
# 1     3185
# 2      627

# Coord similar
# 0    16550
# 5     8736
# 1     3046
# 2      570

In [21]:
CallesAnalise.to_file("D:/JOB/IGEO/2020 - Metrogas - Geocoder/CABA_Checked.geojson", driver='GeoJSON') 

In [ ]:
print(Calles.loc[(Calles.NOM_CALLE == NombreCalles[0]) & (Calles.DESDIZQ == DireccionFin+1), 'Ini_X'])
print(Calles.loc[(Calles.NOM_CALLE == NombreCalles[0]) & (Calles.DESDIZQ == DireccionFin+1), 'Ini_Y'])

In [ ]:
CalleSigX

In [ ]:
Calles.info()